In [1]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time

# Load the YOLO11n-pose model
model = YOLO("yolo11n-pose.pt")

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define keypoint indices for elbows (based on COCO 17-keypoint format)
LEFT_ELBOW = 7
RIGHT_ELBOW = 8

# List to store elbow coordinates and timestamps
data = []

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform pose estimation
    results = model(frame, conf=0.5, classes=[0])  # Detect only person class

    # Get current timestamp
    timestamp = time.time()

    # Process results
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # Shape: (num_persons, 17, 2)
            confidences = result.keypoints.conf.cpu().numpy()  # Shape: (num_persons, 17)

            for person_idx, (kpts, confs) in enumerate(zip(keypoints, confidences)):
                # Extract elbow coordinates
                # if confs[LEFT_ELBOW] > 0.5:  # Check confidence threshold
                #     left_elbow_x, left_elbow_y = kpts[LEFT_ELBOW]
                #     # Draw left elbow keypoint
                #     cv2.circle(frame, (int(left_elbow_x), int(left_elbow_y)), 5, (0, 255, 0), -1)
                    # Log left elbow data
                    # data.append({
                    #     "timestamp": timestamp,
                    #     "person_id": person_idx,
                    #     "joint": "left_elbow",
                    #     "x": left_elbow_x,
                    #     "y": left_elbow_y,
                    #     "confidence": confs[LEFT_ELBOW]
                    # })

                if confs[RIGHT_ELBOW] > 0.5:  # Check confidence threshold
                    right_elbow_x, right_elbow_y = kpts[RIGHT_ELBOW]
                    # Draw right elbow keypoint
                    cv2.circle(frame, (int(right_elbow_x), int(right_elbow_y)), 15, (0, 0, 255), -1)
                    #Log right elbow data
                    # data.append({
                    #     "timestamp": timestamp,
                    #     "person_id": person_idx,
                    #     "joint": "right_elbow",
                    #     "x": right_elbow_x,
                    #     "y": right_elbow_y,
                    #     "confidence": confs[RIGHT_ELBOW]
                    # })

    # Display the frame
    cv2.imshow("Elbow Tracking", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Save elbow data to CSV
# df = pd.DataFrame(data)
# df.to_csv("elbow_movement.csv", index=False)
# print("Elbow movement data saved to 'elbow_movement.csv'")

100%|██████████| 5.97M/5.97M [00:00<00:00, 27.4MB/s]



0: 480x640 2 persons, 116.9ms
Speed: 4.2ms preprocess, 116.9ms inference, 11.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 42.2ms
Speed: 1.5ms preprocess, 42.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 40.7ms
Speed: 1.1ms preprocess, 40.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 42.6ms
Speed: 2.0ms preprocess, 42.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 45.1ms
Speed: 2.1ms preprocess, 45.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 46.7ms
Speed: 1.2ms preprocess, 46.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 46.8ms
Speed: 1.2ms preprocess, 46.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 48.3ms
Speed: 1.1ms preprocess, 48.3ms inference, 1.0ms postprocess per image at shap

In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time
from collections import deque

# Load the YOLO11n-pose model
model = YOLO("yolo11n-pose.pt")

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define keypoint indices for right arm (COCO 17-keypoint format)
RIGHT_SHOULDER = 6
RIGHT_ELBOW = 8
RIGHT_WRIST = 10

# Parameters for wave detection
WAVE_THRESHOLD = 50  # Pixel threshold for wrist vertical movement
WAVE_WINDOW = 10     # Number of frames to analyze for wave pattern
wrist_history = deque(maxlen=WAVE_WINDOW)  # Store recent wrist y-coordinates

# List to store arm data and wave status
data = []

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform pose estimation
    results = model(frame, conf=0.5, classes=[0])  # Detect only person class

    # Get current timestamp
    timestamp = time.time()

    # Process results
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # Shape: (num_persons, 17, 2)
            confidences = result.keypoints.conf.cpu().numpy()  # Shape: (num_persons, 17)

            for person_idx, (kpts, confs) in enumerate(zip(keypoints, confidences)):
                # Extract right arm keypoints
                shoulder_detected = confs[RIGHT_SHOULDER] > 0.5
                elbow_detected = confs[RIGHT_ELBOW] > 0.5
                wrist_detected = confs[RIGHT_WRIST] > 0.5

                # Initialize variables
                shoulder_x, shoulder_y = None, None
                elbow_x, elbow_y = None, None
                wrist_x, wrist_y = None, None
                is_waving = False

                if shoulder_detected:
                    shoulder_x, shoulder_y = kpts[RIGHT_SHOULDER]
                    cv2.circle(frame, (int(shoulder_x), int(shoulder_y)), 5, (255, 0, 0), -1)
                if elbow_detected:
                    elbow_x, elbow_y = kpts[RIGHT_ELBOW]
                    cv2.circle(frame, (int(elbow_x), int(elbow_y)), 5, (0, 255, 0), -1)
                if wrist_detected:
                    wrist_x, wrist_y = kpts[RIGHT_WRIST]
                    cv2.circle(frame, (int(wrist_x), int(wrist_y)), 5, (0, 0, 255), -1)
                    wrist_history.append(wrist_y)

                # Draw lines between detected keypoints
                if shoulder_detected and elbow_detected:
                    cv2.line(frame, (int(shoulder_x), int(shoulder_y)), 
                            (int(elbow_x), int(elbow_y)), (255, 255, 255), 2)
                if elbow_detected and wrist_detected:
                    cv2.line(frame, (int(elbow_x), int(elbow_y)), 
                            (int(wrist_x), int(wrist_y)), (255, 255, 255), 2)

                # Detect waving based on wrist movement
                if len(wrist_history) == WAVE_WINDOW:
                    y_min = min(wrist_history)
                    y_max = max(wrist_history)
                    if y_max - y_min > WAVE_THRESHOLD:
                        is_waving = True
                        cv2.putText(frame, "Waving!", (int(wrist_x), int(wrist_y) - 10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                # # Log right arm data
                # if any([shoulder_detected, elbow_detected, wrist_detected]):
                #     data.append({
                #         "timestamp": timestamp,
                #         "person_id": person_idx,
                #         "shoulder_x": shoulder_x,
                #         "shoulder_y": shoulder_y,
                #         "shoulder_conf": confs[RIGHT_SHOULDER] if shoulder_detected else None,
                #         "elbow_x": elbow_x,
                #         "elbow_y": elbow_y,
                #         "elbow_conf": confs[RIGHT_ELBOW] if elbow_detected else None,
                #         "wrist_x": wrist_x,
                #         "wrist_y": wrist_y,
                #         "wrist_conf": confs[RIGHT_WRIST] if wrist_detected else None,
                #         "is_waving": is_waving
                #     })

    # Display the frame
    cv2.imshow("Right Arm Wave Tracking", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Save data to CSV
# df = pd.DataFrame(data)
# df.to_csv("right_arm_wave.csv", index=False)
# print("Right arm movement data saved to 'right_arm_wave.csv'")


0: 480x640 1 person, 56.3ms
Speed: 1.8ms preprocess, 56.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 40.2ms
Speed: 1.3ms preprocess, 40.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 42.5ms
Speed: 1.9ms preprocess, 42.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 46.2ms
Speed: 1.1ms preprocess, 46.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 38.0ms
Speed: 1.9ms preprocess, 38.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 38.1ms
Speed: 1.1ms preprocess, 38.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 45.8ms
Speed: 2.1ms preprocess, 45.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 42.6ms
Speed: 1.1ms preprocess, 42.6ms inference, 0.9ms postprocess per image at shape (1, 3, 48

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

: 

In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import time
from collections import deque

# Load the YOLO11n-pose model
model = YOLO("yolo11n-pose.pt")

# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define keypoint indices for right arm (COCO 17-keypoint format)
RIGHT_SHOULDER = 6
RIGHT_ELBOW = 8
RIGHT_WRIST = 10

# Parameters for wave and horizontal movement detection
WAVE_THRESHOLD = 50  # Pixel threshold for wrist vertical movement
MOVEMENT_THRESHOLD = 50  # Pixel threshold for wrist horizontal movement
WINDOW_SIZE = 10     # Number of frames to analyze for patterns
wrist_y_history = deque(maxlen=WINDOW_SIZE)  # Store recent wrist y-coordinates
wrist_x_history = deque(maxlen=WINDOW_SIZE)  # Store recent wrist x-coordinates

# List to store arm data and movement status
data = []

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform pose estimation
    results = model(frame, conf=0.5, classes=[0])  # Detect only person class

    # Get current timestamp
    timestamp = time.time()

    # Process results
    for result in results:
        if hasattr(result, 'keypoints') and result.keypoints is not None:
            keypoints = result.keypoints.xy.cpu().numpy()  # Shape: (num_persons, 17, 2)
            confidences = result.keypoints.conf.cpu().numpy()  # Shape: (num_persons, 17)

            for person_idx, (kpts, confs) in enumerate(zip(keypoints, confidences)):
                # Extract right arm keypoints
                shoulder_detected = confs[RIGHT_SHOULDER] > 0.5
                elbow_detected = confs[RIGHT_ELBOW] > 0.5
                wrist_detected = confs[RIGHT_WRIST] > 0.5

                # Initialize variables
                shoulder_x, shoulder_y = None, None
                elbow_x, elbow_y = None, None
                wrist_x, wrist_y = None, None
                is_waving = False
                movement_direction = None  # None, "Left", or "Right"

                if shoulder_detected:
                    shoulder_x, shoulder_y = kpts[RIGHT_SHOULDER]
                    cv2.circle(frame, (int(shoulder_x), int(shoulder_y)), 5, (255, 0, 0), -1)
                if elbow_detected:
                    elbow_x, elbow_y = kpts[RIGHT_ELBOW]
                    cv2.circle(frame, (int(elbow_x), int(elbow_y)), 5, (0, 255, 0), -1)
                if wrist_detected:
                    wrist_x, wrist_y = kpts[RIGHT_WRIST]
                    cv2.circle(frame, (int(wrist_x), int(wrist_y)), 5, (0, 0, 255), -1)
                    wrist_y_history.append(wrist_y)
                    wrist_x_history.append(wrist_x)

                # Draw lines between detected keypoints
                if shoulder_detected and elbow_detected:
                    cv2.line(frame, (int(shoulder_x), int(shoulder_y)), 
                             (int(elbow_x), int(elbow_y)), (255, 255, 255), 2)
                if elbow_detected and wrist_detected:
                    cv2.line(frame, (int(elbow_x), int(elbow_y)), 
                             (int(wrist_x), int(wrist_y)), (255, 255, 255), 2)

                # Detect waving based on wrist vertical movement
                if len(wrist_y_history) == WINDOW_SIZE:
                    y_min = min(wrist_y_history)
                    y_max = max(wrist_y_history)
                    if y_max - y_min > WAVE_THRESHOLD:
                        is_waving = True
                        cv2.putText(frame, "Waving!", (int(wrist_x), int(wrist_y) - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                # Detect left or right movement based on wrist horizontal movement
                if len(wrist_x_history) == WINDOW_SIZE:
                    x_min = min(wrist_x_history)
                    x_max = max(wrist_x_history)
                    x_diff = x_max - x_min
                    if x_diff > MOVEMENT_THRESHOLD:
                        # Determine direction based on the most recent x-coordinates
                        if wrist_x_history[-1] < wrist_x_history[0]:
                            movement_direction = "Left"
                        else:
                            movement_direction = "Right"
                        cv2.putText(frame, f"Moving {movement_direction}", 
                                    (int(wrist_x), int(wrist_y) - 30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

                # Log right arm data
                # if any([shoulder_detected, elbow_detected, wrist_detected]):
                #     data.append({
                #         "timestamp": timestamp,
                #         "person_id": person_idx,
                #         "shoulder_x": shoulder_x,
                #         "shoulder_y": shoulder_y,
                #         "shoulder_conf": confs[RIGHT_SHOULDER] if shoulder_detected else None,
                #         "elbow_x": elbow_x,
                #         "elbow_y": elbow_y,
                #         "elbow_conf": confs[RIGHT_ELBOW] if elbow_detected else None,
                #         "wrist_x": wrist_x,
                #         "wrist_y": wrist_y,
                #         "wrist_conf": confs[RIGHT_WRIST] if wrist_detected else None,
                #         "is_waving": is_waving,
                #         "movement_direction": movement_direction
                #     })

    # Display the frame
    cv2.imshow("Right Arm Movement Tracking", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Save data to CSV
# df = pd.DataFrame(data)
# df.to_csv("right_arm_movement.csv", index=False)
# print("Right arm movement data saved to 'right_arm_movement.csv'")